Author: Kris Su

Dataset handler imports
- geopandas library will read the polygon files defined in geojson
- xarray library is used for netCDF processing

In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd

Visualization components import

In [28]:
from dash import Dash, html
from plotly.data import iris
from plotly.colors import sequential
import sys

import holoviews as hv
from holoviews.plotting.plotly.dash import to_dash
from holoviews.operation.datashader import datashade, rasterize, shade

In [34]:
# Load dataset
with xr.open_dataset(r"C:\Users\Kris\Documents\amazonforcast\data\prakrut\output\LIS_HIST_Forecast_June_02_to_05_mean.nc") as ds:
    df_original = ds["Rainf_tavg"].isel(time = 1).to_dataframe().reset_index().dropna()
    ds.close # close the netCDF files to free up RAM spaces
    # the following lines are included for debugging and book keeping purposes
    print(type(df_original))
    print(f"{sys.getsizeof(df_original)} in bytes")

df_original

<class 'pandas.core.frame.DataFrame'>
10607252 in bytes


,north_south,east_west,time,Rainf_tavg
237,-21.0,-70.132018,2024-06-03,6.324491e-07
238,-21.0,-70.081942,2024-06-03,6.176514e-07
239,-21.0,-70.031866,2024-06-03,5.941062e-07
240,-21.0,-69.981791,2024-06-03,5.641532e-07
241,-21.0,-69.931715,2024-06-03,5.297748e-07
...,...,...,...,...
356282,6.0,-54.858877,2024-06-03,5.893870e-05
356283,6.0,-54.808801,2024-06-03,5.954330e-05
356284,6.0,-54.758725,2024-06-03,6.041464e-05
356285,6.0,-54.708649,2024-06-03,6.139366e-05


In [38]:
# Convert from lon/lat to web-mercator easting/northing coordinates
df_original["easting"], df_original["northing"] = hv.Tiles.lon_lat_to_easting_northing(
df_original["east_west"], df_original["north_south"]
)

df_original #df_original["easting"], df_original["northing"]

,north_south,east_west,time,Rainf_tavg,easting,northing
237,-21.0,-70.132018,2024-06-03,6.324491e-07,-7.807061e+06,-2.391879e+06
238,-21.0,-70.081942,2024-06-03,6.176514e-07,-7.801486e+06,-2.391879e+06
239,-21.0,-70.031866,2024-06-03,5.941062e-07,-7.795912e+06,-2.391879e+06
240,-21.0,-69.981791,2024-06-03,5.641532e-07,-7.790337e+06,-2.391879e+06
241,-21.0,-69.931715,2024-06-03,5.297748e-07,-7.784763e+06,-2.391879e+06
...,...,...,...,...,...,...
356282,6.0,-54.858877,2024-06-03,5.893870e-05,-6.106862e+06,6.691411e+05
356283,6.0,-54.808801,2024-06-03,5.954330e-05,-6.101288e+06,6.691411e+05
356284,6.0,-54.758725,2024-06-03,6.041464e-05,-6.095713e+06,6.691411e+05
356285,6.0,-54.708649,2024-06-03,6.139366e-05,-6.090139e+06,6.691411e+05


In [57]:
# Duplicate carshare dataframe with Gaussian noise to produce a larger dataframe
df = pd.concat([df_original]*4)
df["easting"] = df["easting"] + np.random.randn(len(df)) * 400
df["northing"] = df["northing"] + np.random.randn(len(df)) * 400

In [58]:
# Build Dataset and graphical elements
# dataset = hv.Dataset(df_original)
points = hv.Points(df,["easting", "northing"]).opts(color="Rainf_tavg")
datashade(points)

BokehModel(combine_events=True, render_bundle={'docs_json': {'c2ec431d-faa6-41d5-830f-e6183702c8ae': {'version…

In [48]:
tiles = hv.Tiles().opts(mapboxstyle="open-street-map")


In [27]:

overlay = tiles * datashade(points)

overlay.opts(
title="Mapbox Datashader with %d points" % len(df_original),
width=800,
height=500
)

# Build app
app = Dash(__name__)
components = to_dash(app, [overlay], reset_button=True)

app.layout = html.Div(components.children)

if __name__ == "__main__":
    app.run(debug=False)

ValueError: Elements of type <class 'holoviews.element.raster.HeatMap'> cannot be overlaid with Tiles elements using the plotly backend